# Mapping the tracks with water levels

This notebook show the way to map tracks with the water level information.

1. Mapping one track
2. Mapping multiple tracks

In later notebook, the temporal water level changes would be dealt.

## Setting

Required setting in the previous notebook.

In [ ]:
import pandas as pd
from pathlib import Path
import h5py
import re
import numpy as np

In [ ]:
##### load files
## set the directory
data_home = Path('/home/jovyan/ICESat_water_level/extraction/download/')
## list them up and check them
files= list(data_home.glob('*.h5'))
## choose a file as an example
file_latest = files[37]

##### function of reading alt13
def alt13_to_df(filename, beam):    
    f = h5py.File(filename, 'r')
    f_beam = f[beam]
    lat = f_beam['segment_lat'][:]
    long = f_beam['segment_lon'][:]
    ws = f_beam['ht_water_surf'][:]
    ws_sd = f_beam['stdev_water_surf'][:]
    ws_err = f_beam['err_ht_water_surf'][:]
    ortho = f_beam['ht_ortho'][:]
    wd = f_beam['water_depth'][:]
    alt13_df = pd.DataFrame({'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                            'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                            'WaterD':wd})
    return alt13_df

Check whether it works well.

In [ ]:
gt2r = alt13_to_df(file_latest,'gt2r')
print(gt2r)

## Step 1: Mapping one track with water level information

We will use `surface water hegiht` and `Orthometric height` as water level infromation.

### Showing the water level information.

In [ ]:
print('Surface water height - Min. height:', gt2r['SurfaceH'].min(),'(m)', 'Max. height:', gt2r['SurfaceH'].max(),'(m)')
print('Orthometric height - Min. height:', gt2r['OrthoH'].min(),'(m)', 'Max. height:', gt2r['OrthoH'].max(),'(m)')

In [ ]:
import h5py
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib as mpl        
import cartopy.io.img_tiles as cimgt

In [ ]:
### bounds of Tonle Sap Lake
sp_ex = [103.643, 104.667, 12.375, 13.287]

In [ ]:
## function for mapping the water level 
def ic2_map_wl(file, beam, bound, water_levels):
    ic2 = alt13_to_df(file,beam)
    fig = plt.figure(figsize=(9,10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent(bound, crs=ccrs.PlateCarree())
    plt.scatter(ic2['Longitude'], ic2['Latitude'], s=10, c=ic2[water_levels], alpha=.7, transform=ccrs.PlateCarree(), cmap='terrain')
    plt.colorbar(fraction=0.0320, pad=0.02, label='Elevation (m)')

    ##load Google Sat.Map
    #request = cimgt.GoogleTiles(style='satellite')
    request = cimgt.Stamen('terrain-background')
    ax.add_image(request, 7)

In [ ]:
ic2_map_wl(file_latest,'gt2r',sp_ex,'OrthoH')

In [ ]:
ic2_map_wl(file_latest,'gt2r',sp_ex,'SurfaceH')

### Showing water level for all beams

In [ ]:
## function for mapping the water level from every beam
def ic2_map_wl_beams(file, bound, water_levels):
    fig = plt.figure(figsize=(9,10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent(bound, crs=ccrs.PlateCarree())
    beam = ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r']
    for i in beam:
        ic2 = alt13_to_df(file,i)
        plt.scatter(ic2['Longitude'], ic2['Latitude'], s=10, c=ic2[water_levels], alpha=.7, transform=ccrs.PlateCarree(), cmap='terrain')
    plt.colorbar(fraction=0.0320, pad=0.02, label='Elevation (m)')

    ##load Google Sat.Map
    #request = cimgt.GoogleTiles(style='satellite')
    request = cimgt.Stamen('terrain-background')
    ax.add_image(request, 7)

In [ ]:
ic2_map_wl_beams(file_latest,sp_ex,'OrthoH')

In [ ]:
ic2_map_wl_beams(file_latest,sp_ex,'SurfaceH')

## Step 2: Mapping multiple tracks with water level information

### Mapping the tracks